In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()


In [26]:
ph_path = PROJECT_ROOT / "powerbi/exports/PH_XAU_8p88y_Rolling.csv"

df_ph = pd.read_csv(ph_path, parse_dates=["date"])
df_ph = df_ph.set_index("date").sort_index()

df_ph.head()


,cycle_strength,window,m,tau,representation,cyclic_regime
date,,,,,,
1819-12-31,0.035242,107,12,3,log_price,Non-Cyclic
1820-03-31,0.035242,107,12,3,log_price,Non-Cyclic
1820-03-31,0.000000,107,12,3,log_returns,Non-Cyclic
1820-06-30,0.000000,107,12,3,log_returns,Non-Cyclic
1820-06-30,0.035242,107,12,3,log_price,Non-Cyclic


In [27]:
df_ph.info()
df_ph.head()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2577 entries, 1819-12-31 to 2025-12-31
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cycle_strength  2577 non-null   float64
 1   window          2577 non-null   int64  
 2   m               2577 non-null   int64  
 3   tau             2577 non-null   int64  
 4   representation  2577 non-null   object 
 5   cyclic_regime   2577 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 140.9+ KB


,cycle_strength,window,m,tau,representation,cyclic_regime
date,,,,,,
1819-12-31,0.035242,107,12,3,log_price,Non-Cyclic
1820-03-31,0.035242,107,12,3,log_price,Non-Cyclic
1820-03-31,0.000000,107,12,3,log_returns,Non-Cyclic
1820-06-30,0.000000,107,12,3,log_returns,Non-Cyclic
1820-06-30,0.035242,107,12,3,log_price,Non-Cyclic


In [43]:
df_ph = df_ph[df_ph["representation"] == "log_price"].copy()
df_ph["cyclic_regime"].value_counts()


cyclic_regime
Non-Cyclic    1223
Cyclic          66
Name: count, dtype: int64

In [44]:
def load_macro(filename, value_col):
    path = PROJECT_ROOT / "data/raw" / filename
    df = pd.read_csv(path, parse_dates=["Date"])

    # Normalize to end-of-month to match XAU
    df["Date"] = df["Date"] + pd.offsets.MonthEnd(0)

    return df.set_index("Date")[value_col]



In [45]:
cpi = load_macro("CPI_Monthly.csv", "CPI")
m2  = load_macro("M2_Monthly.csv", "M2")
ff  = load_macro("FEDFUNDS_Monthly.csv", "FedFunds")


In [46]:
df = df_ph.copy()

df["CPI"] = cpi
df["M2"] = m2
df["FedFunds"] = ff

df.tail()


,cycle_strength,window,m,tau,representation,cyclic_regime,CPI,M2,FedFunds
date,,,,,,,,,
2025-08-31,0.031089,107,12,3,log_price,Non-Cyclic,323.364,22108.3,4.33
2025-09-30,0.031089,107,12,3,log_price,Non-Cyclic,324.368,22212.4,4.22
2025-10-31,0.031089,107,12,3,log_price,Non-Cyclic,NaN,22298.0,4.09
2025-11-30,0.031089,107,12,3,log_price,Non-Cyclic,325.031,22322.4,3.88
2025-12-31,0.031089,107,12,3,log_price,Non-Cyclic,NaN,NaN,NaN


In [47]:
df["inflation_yoy"] = df["CPI"].pct_change(12)
df["inflation_accel"] = df["inflation_yoy"].diff()

df["inflation_regime"] = np.where(
    df["inflation_accel"] > 0,
    "Accelerating",
    "Decelerating"
)


/var/folders/qt/8016q_0d4kb4ly1txywph44r0000gn/T/ipykernel_22043/1899793426.py:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["inflation_yoy"] = df["CPI"].pct_change(12)


In [49]:
df["m2_yoy"] = df["M2"].pct_change(12)

df["liquidity_regime"] = np.where(
    df["m2_yoy"] > df["m2_yoy"].median(),
    "Loose",
    "Tight"
)


/var/folders/qt/8016q_0d4kb4ly1txywph44r0000gn/T/ipykernel_22043/3038226972.py:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["m2_yoy"] = df["M2"].pct_change(12)


In [50]:
df["rate_regime"] = np.where(
    df["FedFunds"] > df["FedFunds"].median(),
    "High",
    "Low"
)


In [51]:
df_cond = df.dropna(
    subset=[
        "inflation_regime",
        "liquidity_regime",
        "rate_regime",
        "cyclic_regime"
    ]
)

df_cond.shape


(1289, 15)

In [52]:
pd.crosstab(
    df_cond["inflation_regime"],
    df_cond["cyclic_regime"],
    normalize="index"
)

pd.crosstab(
    df_cond["liquidity_regime"],
    df_cond["cyclic_regime"],
    normalize="index"
)

pd.crosstab(
    df_cond["rate_regime"],
    df_cond["cyclic_regime"],
    normalize="index"
)




cyclic_regime,Cyclic,Non-Cyclic
rate_regime,,
High,0.064343,0.935657
Low,0.045852,0.954148


In [53]:
pd.crosstab(
    [df_cond["inflation_regime"], df_cond["liquidity_regime"]],
    df_cond["cyclic_regime"],
    normalize="index"
)


cyclic_regime                        Cyclic  Non-Cyclic
inflation_regime liquidity_regime                      
Accelerating     Loose             0.000000    1.000000
                 Tight             0.094737    0.905263
Decelerating     Loose             0.000000    1.000000
                 Tight             0.064953    0.935047

In [54]:
df_cond.groupby("inflation_regime")["cycle_strength"].mean()
df_cond.groupby("liquidity_regime")["cycle_strength"].mean()
df_cond.groupby("rate_regime")["cycle_strength"].mean()


rate_regime
High    0.085359
Low     0.050316
Name: cycle_strength, dtype: float64

In [55]:
out_path = PROJECT_ROOT / "powerbi/exports/PH_XAU_8p88y_MacroConditioned.csv"
df_cond.reset_index().to_csv(out_path, index=False)

out_path


PosixPath('/Users/Rod/Desktop/cycle-tda-lab/powerbi/exports/PH_XAU_8p88y_MacroConditioned.csv')

In [56]:
df[["CPI", "M2", "FedFunds"]].dropna().index.max()

Timestamp('2025-11-30 00:00:00')